Enfoque del dataset de noticias falsas basado en algoritmos clásicos de aprendizaje automático

In [ ]:
### IMPORTS ###
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from keras.utils import to_categorical

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Cargar Dataset

In [ ]:
### GOOGLE DRIVE ###
from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/My Drive/TFM/Data/Fakeddit/"

Mounted at /content/drive


In [ ]:
### PARTICIÓN ###
df = pd.read_csv(PATH + "multimodal_train.tsv",  sep='\t', on_bad_lines='skip')
df = df[['clean_title', '6_way_label']]
df = df.rename(columns={"clean_title": "Text", "6_way_label": "Label"})
df.fillna(" ", inplace=True)

X_train = df['Text']
y_train = df['Label']

X_train, X_aux, y_train, y_aux = train_test_split(X_train, y_train, test_size=0.98, random_state=55, stratify=y_train)

df = pd.read_csv(PATH + "multimodal_test_public.tsv",  sep='\t', on_bad_lines='skip')
df = df[['clean_title', '6_way_label']]
df = df.rename(columns={"clean_title": "Text", "6_way_label": "Label"})
df.fillna(" ", inplace=True)

X_test = df['Text']
y_test = df['Label']

X_test, X_aux, y_test, y_aux = train_test_split(X_test, y_test, test_size=0.98, random_state=55, stratify=y_test)

print('Tamaño conjunto de Entrenamiento:', len(X_train))
print('Tamaño conjunto de Evaluación:', len(X_test))

Tamaño conjunto de Entrenamiento: 11280
Tamaño conjunto de Evaluación: 1186


# Limpieza y Representación de Textos

In [ ]:
### LIMPIEZA DE TEXTOS ###
stopwords_en = stopwords.words("english")
def clean_text(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords_en]
    # usar los stems
    tokens = [PorterStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 2 caráceres
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    min_length = 2
    p = re.compile('^[a-zA-Z]+$');
    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)

    return filtered_tokens

# Data Augmentation

No es necesario ya que el dataset es suficientemente grande

# Label Encoding

No es necesario hacer un encoding ya que viene etiquetado de manera numérica

# Bolsa de Palabras

In [ ]:
### BOLSA DE PALABRAS ###
X_train = X_train.tolist()
X_test = X_test.tolist()

# entrenamos un modelo de bolsa de palabras
bow = CountVectorizer(analyzer=clean_text).fit(X_train)
# transformamos el conjunto de entrenamiento a bolsa de palabras
X_train_bow = bow.transform(X_train)
# transformamos el conjunto de evaluación a bolsa de palabras
X_test_bow = bow.transform(X_test)

print("Tamaño del vocabulario: ", len(bow.vocabulary_))

Tamaño del vocabulario:  11532


# TF-IDF

In [ ]:
### TF-IDF ###
# entrenamos un modelo tf-idf
tfidf_transformer = TfidfTransformer().fit(X_train_bow)
# transformamos el conjunto de entrenamiento
X_train_tfidf = tfidf_transformer.transform(X_train_bow)
# transformamos el conjunto de entrenamiento
X_test_tfidf = tfidf_transformer.transform(X_test_bow)

# Clasificación Clásica
Se crea un pipeline que ejecuta una secuencia de procesos:


1.   La representación de los textos en bolsa de palabras (CountVectorizer), que recibe como entrada los textos, y se les aplica dentro del CountVectorizer la función clean_text para limpiarlos y reducir el ruido.
2.   La representación en tf-idf (TfidfTransformer), recibe como entrada la salida del proceso 1, y produce los vectores tf-idf.
3. El clasificador SVC, Logistic Regression o Random Forest Clasiffier.

In [ ]:
### PIPELINE SVM ###
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('svm', SVC()),
])

# Parámetros para el algoritmo SVM
grid_params_svm = [{'svm__kernel': ['linear', 'rbf'],
                    'svm__C': [0.1, 1], # [0.1, 1, 10, 100, 1000]
                    'svm__gamma':  [1, 0.1] # [1, 0.1, 0.01, 0.001, 0.0001]
                    }]
gs = GridSearchCV(pipeline, param_grid=grid_params_svm,
                  scoring='accuracy', cv=4, verbose = 1)

# entrenamos el grid
gs.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs.best_params_)
print('Mejor accuracy: %.3f' % gs.best_score_)
print(gs.best_estimator_)

best_svm = gs.best_estimator_
predictions = best_svm.predict(X_test)
print(classification_report(y_test, predictions))

Fitting 4 folds for each of 8 candidates, totalling 32 fits
Los mejores parámetros son : {'svm__C': 1, 'svm__gamma': 1, 'svm__kernel': 'linear'}
Mejor accuracy: 0.593
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7efd400a45e0>)),
                ('tf', TfidfTransformer()),
                ('svm', SVC(C=1, gamma=1, kernel='linear'))])
              precision    recall  f1-score   support

           0       0.58      0.81      0.68       470
           1       0.40      0.06      0.10        70
           2       0.56      0.32      0.41       226
           3       1.00      0.04      0.08        25
           4       0.60      0.64      0.61       349
           5       0.78      0.30      0.44        46

    accuracy                           0.59      1186
   macro avg       0.65      0.36      0.39      1186
weighted avg       0.59      0.59      0.55      1186



In [ ]:
### PIPELINE LOGISTIC REGRESSION ###
pipeline2 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('lr', LogisticRegression(random_state=0)),
])

# Parámetros para el algoritmo Logistic Regression
grid_params_lr = [{'lr__penalty': ['l1', 'l2'],
                    'lr__C': [1.0, 0.5],
                    'lr__solver':  ['liblinear']
                    }]
gs2 = GridSearchCV(pipeline2, param_grid=grid_params_lr,
                  scoring='accuracy', cv=5, verbose = 1)

# entrenamos el grid
gs2.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs2.best_params_)
print('Mejor accuracy: %.3f' % gs2.best_score_)
print(gs2.best_estimator_)

best_svm = gs2.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Los mejores parámetros son : {'lr__C': 1.0, 'lr__penalty': 'l2', 'lr__solver': 'liblinear'}
Mejor accuracy: 0.589
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7efd400a45e0>)),
                ('tf', TfidfTransformer()),
                ('lr', LogisticRegression(random_state=0, solver='liblinear'))])
              precision    recall  f1-score   support

           0       0.57      0.86      0.68       470
           1       1.00      0.01      0.03        70
           2       0.62      0.27      0.38       226
           3       0.00      0.00      0.00        25
           4       0.60      0.63      0.61       349
           5       0.89      0.17      0.29        46

    accuracy                           0.59      1186
   macro avg       0.61      0.32      0.33      1186
weighted avg       0.61      0.59      0.54      1186



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
### PIPELINE RANDOM FOREST ###
pipeline3 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('rfc', RandomForestClassifier(random_state=0)),
])

# Parámetros para el algoritmo Random Forest
grid_params_rfc = [{'rfc__criterion': ['gini', 'entropy'],
                    'rfc__max_depth': [9, 10],
                    'rfc__min_samples_split':  [10]
                    }]
gs3 = GridSearchCV(pipeline3, param_grid=grid_params_rfc,
                  scoring='accuracy', cv=5, verbose = 1)

# entrenamos el grid
gs3.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs3.best_params_)
print('Mejor accuracy: %.3f' % gs3.best_score_)
print(gs3.best_estimator_)

best_svm = gs3.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Los mejores parámetros son : {'rfc__criterion': 'gini', 'rfc__max_depth': 10, 'rfc__min_samples_split': 10}
Mejor accuracy: 0.423
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7efd400a45e0>)),
                ('tf', TfidfTransformer()),
                ('rfc',
                 RandomForestClassifier(max_depth=10, min_samples_split=10,
                                        random_state=0))])
              precision    recall  f1-score   support

           0       0.40      1.00      0.57       470
           1       0.00      0.00      0.00        70
           2       0.00      0.00      0.00       226
           3       0.00      0.00      0.00        25
           4       1.00      0.05      0.10       349
           5       0.00      0.00      0.00        46

    accuracy                           0.41      1186
   macro avg       0.23      0.18      0.11      1186
weighted a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
